In [ ]:
import time
import sys
import ibmiotf.application
import ibmiotf.device
import requests
url = "https://www.fast2sms.com/dev/bulk"

#My IoT platform device's credentials to connect the device and the platform
organization="maql17"
deviceType= "RaspberryPi"
deviceId= "123456"
authMethod= "token"
authToken="12345678"
    
def myCommandCallback(cmd):
        print("Command received : %s !" % cmd.data)  
        
try:
    deviceOptions= {"org": organization, "type": deviceType, "id": deviceId, "auth-method": authMethod, "auth-token": authToken}
    deviceCli= ibmiotf.device.Client(deviceOptions)
except Exception as e:
    print("Caught exception connection device: %s" % str(e))
    sys.exit()

deviceCli.connect()

#   Since we are not using actual hardware to demonstrate. We assume a scenario which contains all the possible outputs.
#   We assume weight of the jar to be 1500 grams.
#   We know the weight of a full cylinder is 30 Kgs. But we must calibrate our device such that it only takes the weight of 
#the gas inside the cylinder and let us know if its empty. So we take the weight of the cylinder to be 15Kgs(gas weight).

jar_weight=1500
cylinder_weight=15
Fan= "OFF"
leak="OFF"
i=0

#   Variables cyl_empty and jar_empty are used as flags. This is to make sure that when we send the alert sms, the alert is sent
# only once until reset. So this prevents resending messages everytime the sensor detects empty in the loop.
cyl_empty=0;
jar_empty=0

while True:
    #We assume the cylinder to be used 0.01 times in every loop. So we can demonstrate the stages of the cylinder and its status
    cylinder_weight=cylinder_weight-0.1;
    
    # One tablespoon is 14 grams. Hence we reduce 14 grams everytime to simulate taking sugar out of a jar with a tablespoon
    jar_weight=jar_weight-14;

    # Depending on the weight of the cylinder in Kgs. The current status of gas is sent ot the user. The condition is as below.
    
    if(cylinder_weight >0 and cylinder_weight<=5):
        status="LOW"
        time.sleep(0.1)
    elif(cylinder_weight >5 and cylinder_weight<=10):
        status="MEDIUM"
    elif(cylinder_weight >10 and cylinder_weight<=15):
        status="HIGH"
    else:
        cylinder_weight=0
        status="EMPTY"
        
        #If the cylinder goes empty, We alert the user by sending an SMS that the cylinder has gone empty using FAST2SMS.
        if(cyl_empty==0):
            payload = "sender_id=FSTSMS&message=Your Cylinder is Empty. Book a new one!&language=english&route=p&numbers=9003191279"
            headers = {'authorization': "64a9QD2NfZVIpxqGwcOtKrBMnFWAzdkmXCjTYoR5yeS107Llb39NAEYamlfeipo4Wvht8qn0QLO2cTzR",
                         'Content-Type': "application/x-www-form-urlencoded",'Cache-Control': "no-cache", }
            response = requests.request("POST", url, data=payload, headers=headers)
            print(response.text)
            print("The cylinder is empty")
            # cyl_empty flag is changed to 1 to prevent repeating the sms until reset.
            cyl_empty=1;
    
    # Depending on the weight of the Jar in grams. The current status of gas is sent ot the user. The condition is as below.
    
    if(jar_weight <300 and jar_weight>=0):
        jar_status="LOW"
        time.sleep(0.1)
    elif(jar_weight >=300 and jar_weight<1000):
        jar_status="MEDIUM"
    elif(jar_weight >=1000 and jar_weight<=1500):
        jar_status="HIGH"
    else:
        jar_weight=0
        jar_status="EMPTY"
        if(jar_empty==0):
            print("The JAR is empty!")
            payload = "sender_id=FSTSMS&message=The content in the jar is empty!. Refill to continue usage.&language=english&route=p&numbers=9003191279"
            headers = {'authorization': "64a9QD2NfZVIpxqGwcOtKrBMnFWAzdkmXCjTYoR5yeS107Llb39NAEYamlfeipo4Wvht8qn0QLO2cTzR",
                         'Content-Type': "application/x-www-form-urlencoded",'Cache-Control': "no-cache", }
            response = requests.request("POST", url, data=payload, headers=headers)
            print(response.text)
            # jar_empty flag is changed to 1 to prevent repeating the sms until reset.
            jar_empty=1;
    
    #Since we are not using hardware to detect the gas leak, we simulate to show this condition is also taken care.
    #We assume that after 50 loops, the gas starts leaking and we alert the user by sending an sms.
    i=i+1
    if(i==50):
        print("Gas leakage detected! Check immediately!")
        payload = "sender_id=FSTSMS&message=Gas leakage detected! Check immediately!&language=english&route=p&numbers=9003191279"
        headers = {'authorization': "64a9QD2NfZVIpxqGwcOtKrBMnFWAzdkmXCjTYoR5yeS107Llb39NAEYamlfeipo4Wvht8qn0QLO2cTzR",
                     'Content-Type': "application/x-www-form-urlencoded",'Cache-Control': "no-cache", }
        response = requests.request("POST", url, data=payload, headers=headers)
        print(response.text)
        
        #If there is gas leak, we turn on the exhaust fan.
        Fan="ON"
        leak="ON"
    
    #We store all our data collected in JSON format in as "data".
    
    data= {'cylinder_weight' : round(cylinder_weight,2), 'status': status,'leak':leak, 'jar_weight': jar_weight, 'jar_status':jar_status,'FAN_STATUS':Fan}
    
    #function to show if the data is published in our iot platform.
    def myOnPublishCallback():
        print(" Published Cylinder_weight = %s " %round(cylinder_weight,2) , "Cylinder_Status=%s " %status,"Gas leakage= %s"%leak,"jar_weight = %s " %jar_weight , "Jar_Status=%s " %jar_status,"FAN_STATUS =%s"%Fan,"\n")
    
    #publishEvent function published the json data to the IBM platform
    success = deviceCli.publishEvent("Smart Kitchen","json", data ,qos=0, on_publish=myOnPublishCallback)
    
    #if we couldnt connect to the IBM platform, not connected to IotF is printed.
    if not success:
        print("Not connected to IoTF")
    time.sleep(0.6)
    deviceCli.commandCallback= myCommandCallback
deviceCli.disconnect()